# Bash Scripting

Task 1: File Management Script Write a Bash script that Creates a directory named "backup" in the user's home directory Copies all .txt files from the current directory into the "backup" directory Appends the current date and time to the filenames of the copied files.

In [ ]:
#!/bin/bash

# Create 'backup' directory in the user's home if not exists
mkdir -p ~/backup

# Loop through all .txt files in current directory
for file in *.txt; do
    # Check if file exists to avoid errors when no txt files
    if [ -f "$file" ]; then
        # Get current date-time
        datetime=$(date +"%Y-%m-%d_%H-%M-%S")
        # Copy file to backup with date-time appended
        cp "$file" ~/backup/"${file%.txt}_$datetime.txt"
    fi
done

echo "Backup completed!"


Task 2: System Health Check Create a script that Checks the system’s CPU and memory usage Reports if the CPU usage is above 80% or if the available memory is below 20% Logs the results to a file named system_health.log.

In [ ]:
#!/bin/bash

# Get CPU usage (extracts idle value, subtracts from 100)
cpu_usage=$(top -bn1 | grep "Cpu(s)" | awk '{print 100 - $8}')

# Get available memory percentage
mem_free=$(free | grep Mem | awk '{printf "%.0f", $7/$2 * 100}')

# Log file
log_file="system_health.log"

# Check conditions and log
{
    echo "==== System Health Check: $(date) ===="
    echo "CPU Usage: $cpu_usage%"
    echo "Free Memory: $mem_free%"

    if (( $(echo "$cpu_usage > 80" | bc -l) )); then
        echo "Warning: CPU usage is above 80%"
    fi

    if [ "$mem_free" -lt 20 ]; then
        echo "Warning: Available memory is below 20%"
    fi

    echo "======================================"
} >> "$log_file"


Task 3: User Account Management Write a script that Reads a list of usernames from a file (e.g., user_list.txt) Creates a new user for each username Generates a random password for each user and saves the username and password to a file named credentials.txt.

In [ ]:
#!/bin/bash

# Check for user_list.txt
if [ ! -f "user_list.txt" ]; then
    echo "user_list.txt not found!"
    exit 1
fi

# Clear credentials file
> credentials.txt

# Loop through usernames and create users
while read -r username; do
    sudo useradd "$username"
    password=$(openssl rand -base64 12)
    echo "$username:$password" | sudo chpasswd
    echo "$username $password" >> credentials.txt
done < user_list.txt

echo "Users created and credentials saved to credentials.txt"


Task 4: Automated Backup Create a script that Takes a directory path as input from the user Compresses the directory into a .tar.gz file Saves the compressed file with a name that includes the current date (e.g., backup_2023-08-20.tar.gz).

In [ ]:
#!/bin/bash

# Ask for directory path
read -p "Enter directory path to backup: " dir_path

# Check if directory exists
if [ ! -d "$dir_path" ]; then
    echo "Directory does not exist!"
    exit 1
fi

# Get current date
date_now=$(date +%Y-%m-%d)

# Create compressed backup file
tar -czf "backup_$date_now.tar.gz" "$dir_path"

echo "Backup completed: backup_$date_now.tar.gz"


Task 5: Simple To-Do List Create a Bash script that Implements a simple command-line to-do list Allows the user to add tasks, view tasks, and remove tasks Saves the tasks to a file (e.g., todo.txt).

In [ ]:
#!/bin/bash

todo_file="todo.txt"

# Ensure todo file exists
touch "$todo_file"

while true; do
    echo "1. Add Task"
    echo "2. View Tasks"
    echo "3. Remove Task"
    echo "4. Exit"
    read -p "Choose an option: " choice

    case $choice in
        1)
            read -p "Enter task: " task
            echo "$task" >> "$todo_file"
            ;;
        2)
            echo "To-Do List:"
            nl "$todo_file"
            ;;
        3)
            nl "$todo_file"
            read -p "Enter task number to remove: " num
            sed -i "${num}d" "$todo_file"
            ;;
        4)
            exit 0
            ;;
        *)
            echo "Invalid option."
            ;;
    esac
done


Task 6: Automated Software Installation Write a script that Reads a list of software package names from a file (e.g., packages.txt) Installs each package using the appropriate package manager (apt, yum, etc.) Logs the installation status of each package.

In [ ]:
#!/bin/bash

# Detect package manager
if command -v apt &> /dev/null; then
    pkg_mgr="sudo apt install -y"
elif command -v yum &> /dev/null; then
    pkg_mgr="sudo yum install -y"
else
    echo "Unsupported package manager!"
    exit 1
fi

# Log file
log_file="install_log.txt"

# Check if packages.txt exists
if [ ! -f "packages.txt" ]; then
    echo "packages.txt not found!"
    exit 1
fi

# Install packages
while read -r package; do
    if $pkg_mgr "$package" &> /dev/null; then
        echo "$package installation successful." >> "$log_file"
    else
        echo "$package installation failed." >> "$log_file"
    fi
done < packages.txt

echo "Installation completed. Log saved to $log_file."


Task 7: Text File Processing Create a script that Takes a text file as input Counts and displays the number of lines, words, and characters in the file Finds and displays the longest word in the file.

In [ ]:
#!/bin/bash

# Get input file
read -p "Enter text file path: " file

# Check file existence
if [ ! -f "$file" ]; then
    echo "File not found!"
    exit 1
fi

# Count lines, words, characters
lines=$(wc -l < "$file")
words=$(wc -w < "$file")
chars=$(wc -m < "$file")

# Find longest word
longest_word=$(tr -s '[:space:]' '\n' < "$file" |
               awk '{ if (length > max) { max = length; longest = $0 } } END
               { print longest }')

# Display results
echo "Lines: $lines"
echo "Words: $words"
echo "Characters: $chars"
echo "Longest Word: $longest_word"
